In [1]:
import warp as wp
import numpy as np
import trimesh
import polyscope as ps
import gpytoolbox as gpy

from warp.sim.collide import TriMeshCollisionDetector

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.


In [2]:
device = wp.get_device('cuda')
# if you don't have a CUDA-compatible GPU try switching to CPU
# device = wp.get_device('cpu')

Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.6
   Devices:
     "cpu"      : "Intel64 Family 6 Model 186 Stepping 2, GenuineIntel"
     "cuda:0"   : "NVIDIA GeForce RTX 3050 6GB Laptop GPU" (6 GiB, sm_86, mempool enabled)
   Kernel cache:
     C:\Users\joana\AppData\Local\NVIDIA\warp\Cache\1.8.0


In [3]:
mesh = trimesh.load("C:/Users/joana/Downloads/Task_1_Bounds_computation/Triangular_Meshes/A00000500.ply")

In [4]:
def viz_mesh():
    V, F = gpy.read_mesh('C:/Users/joana/Downloads/Task_1_Bounds_computation/Triangular_Meshes/A00000500.ply')
    ps.init()
    ps.register_surface_mesh('cube', V, F)
    # pc = ps.register_point_cloud('vertices', V)
    # pc.add_scalar_quantity("vertex_index", np.arange(V.shape[0]), enabled=True, datatype='categorical')
    ps.screenshot("trimesh.png")
    ps.show()
    return V, F
V, F = viz_mesh()


In [5]:
builder = wp.sim.ModelBuilder()

# converts vertex data from an array to warp's vec3 type
vertices = [wp.vec3(mesh.vertices[i,:]) for i in range(mesh.vertices.shape[0])]

# sets up cloth simulation parameters
builder.add_cloth_mesh(
            pos=wp.vec3(0.0, 0.0, 0.0), # position of the mesh
            rot=wp.quat_identity(), # rotation of the mesh
            scale=1.0, # size
            vertices=vertices,
            indices=mesh.faces.reshape(-1),
            vel=wp.vec3(0.0, 0.0, 0.0),
            density=0.02, #mass per volume
            tri_ke=1.0e5, # stiffness of the triangles
            tri_ka=1.0e5,
            tri_kd=2.0e-6,
            edge_ke=10,
)
builder.color()
model = builder.finalize()

Warp UserWarning: The graph is not optimizable anymore, terminated with a max/min ratio: 2.0 without reaching the target ratio: 1.1
Module warp.sim.graph_coloring 3c9ae71 load on device 'cpu' took 1.85 ms  (cached)


In [6]:
# to access ForceElementAdjacencyInfo, you need to construct a VBDIntegrator (you dont need to understand what it is)
# variational block descent
vbd_integrator = wp.sim.VBDIntegrator(model)

Module warp.sim.integrator_vbd 715ddb2 load on device 'cpu' took 29.05 ms  (cached)


In [7]:
colision_detector = TriMeshCollisionDetector(model)

Module warp.sim.collide 3b78b77 load on device 'cuda:0' took 54.91 ms  (cached)


In [8]:
# configuring the collision detection system with specific distance thresholds
#vertex detection within 5 units of triangle surfaces
colision_detector.vertex_triangle_collision_detection(5.0)

# detects when cloth edges come within 5 units
colision_detector.edge_edge_collision_detection(5.0)

In [9]:
# d^v_{min}
print(colision_detector.vertex_colliding_triangles_min_dist)
# d^E_{min}
print(colision_detector.edge_colliding_edges_min_dist)
# d^T_{min}
print(colision_detector.triangle_colliding_vertices_min_dist)

[2.0408168 1.3784707 1.7306733 ... 1.7102327 1.3814771 2.0408168]
[1.3784707 1.8084555 1.514926  ... 1.3814771 2.012112  1.515333 ]
[1.4946659 1.3784707 1.6526986 ... 1.6386598 1.3814771 1.5250734]


In [10]:
from warp.sim.integrator_vbd import get_vertex_num_adjacent_edges, get_vertex_adjacent_edge_id_order, get_vertex_num_adjacent_faces, get_vertex_adjacent_face_id_order, ForceElementAdjacencyInfo
# how to iterate over neighbor elements
@wp.kernel
def iterate_vertex_neighbor_primitives(
    adjacency: ForceElementAdjacencyInfo
):
    particle_idx = wp.tid()

    # iterating over neighbor faces
    num_adj_faces = get_vertex_num_adjacent_faces(adjacency, particle_idx)
    for face_counter in range(num_adj_faces):
        adj_face_idx, vertex_order = get_vertex_adjacent_face_id_order(adjacency, particle_idx, face_counter)
    # iterating over neighbor edges
    num_adj_edges = get_vertex_num_adjacent_edges(adjacency, particle_idx)
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = get_vertex_adjacent_edge_id_order(adjacency, particle_idx, edge_counter)

wp.launch(
    iterate_vertex_neighbor_primitives,
    dim=model.particle_count,
    inputs=[vbd_integrator.adjacency],
    device=device
)

Module __main__ 3dd20ff load on device 'cuda:0' took 27.33 ms  (cached)


In [11]:
# your turn: you need to:
# Implement conservative bounds computation using the instructions provided above
# it must be implemented using @warp.kernel to maximize efficiency

In [ ]:
import warp as wp
from warp.sim.integrator_vbd import get_vertex_num_adjacent_edges, get_vertex_adjacent_edge_id_order, get_vertex_num_adjacent_faces, get_vertex_adjacent_face_id_order, ForceElementAdjacencyInfo

@wp.kernel
def compute_conservative_bounds(
    adjacency: ForceElementAdjacencyInfo,
    positions: wp.array(dtype=wp.vec3),
    vertex_colliding_triangles_min_dist: wp.array(dtype=wp.float32),
    edge_colliding_edges_min_dist: wp.array(dtype=wp.float32),
    triangle_colliding_vertices_min_dist: wp.array(dtype=wp.float32),
    gamma_p: wp.float32,
    conservative_bounds: wp.array(dtype=wp.float32)
):
    """
    Compute conservative bounds for each vertex according to equations (21)-(26).
    
    b_v = gamma_p * min(d_min_v, d^E_min_v, d^T_min_v)
    
    where:
    - d_min_v: minimal distance to all facets that do not include v
    - d^E_min_v: minimal value of v's neighbor edges' minimal distances to all other edges
    - d^T_min_v: minimal value of v's neighbor facets' minimal distances to all other vertices
    """
    vertex_idx = wp.tid()
    
    # Initialize with large values
    d_min_v = wp.float32(1e6)
    d_E_min_v = wp.float32(1e6)
    d_T_min_v = wp.float32(1e6)
    
    # Get vertex position
    v_pos = positions[vertex_idx]
    
    # Compute d_min_v: minimal distance to all facets that do not include vertex_idx
    # This is provided by the collision detector for vertex-triangle collisions
    d_min_v = vertex_colliding_triangles_min_dist[vertex_idx]
    
    # Compute d^E_min_v: minimal value of neighbor edges' minimal distances to other edges
    num_adj_edges = get_vertex_num_adjacent_edges(adjacency, vertex_idx)
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = get_vertex_adjacent_edge_id_order(adjacency, vertex_idx, edge_counter)
        
        # Get the minimal distance for this edge to all other edges
        edge_min_dist = edge_colliding_edges_min_dist[edge_idx]
        
        # Update d^E_min_v with minimum
        if edge_min_dist < d_E_min_v:
            d_E_min_v = edge_min_dist
    
    # Compute d^T_min_v: minimal value of neighbor facets' minimal distances to other vertices
    num_adj_faces = get_vertex_num_adjacent_faces(adjacency, vertex_idx)
    for face_counter in range(num_adj_faces):
        adj_face_idx, vertex_order = get_vertex_adjacent_face_id_order(adjacency, vertex_idx, face_counter)
        
        # Get the minimal distance for this face to all other vertices
        face_min_dist = triangle_colliding_vertices_min_dist[adj_face_idx]
        
        # Update d^T_min_v with minimum
        if face_min_dist < d_T_min_v:
            d_T_min_v = face_min_dist
    
    # Compute conservative bound: b_v = gamma_p * min(d_min_v, d^E_min_v, d^T_min_v)
    min_distance = wp.min(d_min_v, wp.min(d_E_min_v, d_T_min_v))
    conservative_bounds[vertex_idx] = gamma_p * min_distance

# Usage example:
def compute_vertex_conservative_bounds(model, vbd_integrator, collision_detector, gamma_p=0.5):
    """
    Compute conservative bounds for all vertices in the model.
    
    Args:
        model: The simulation model
        vbd_integrator: VBD integrator containing adjacency information
        collision_detector: Collision detector with precomputed minimal distances
        gamma_p: Relaxation parameter (0 < gamma_p < 0.5)
    
    Returns:
        Array of conservative bounds for each vertex
    """
    device = wp.get_device('cuda')
    
    # Allocate output array
    conservative_bounds = wp.zeros(model.particle_count, dtype=wp.float32, device=device)
    
    # Launch kernel
    wp.launch(
        compute_conservative_bounds,
        dim=model.particle_count,
        inputs=[
            vbd_integrator.adjacency,
            model.particle_q,  # positions
            collision_detector.vertex_colliding_triangles_min_dist,
            collision_detector.edge_colliding_edges_min_dist,
            collision_detector.triangle_colliding_vertices_min_dist,
            gamma_p
        ],
        outputs=[conservative_bounds],
        device=device
    )
    
    return conservative_bounds


In [23]:
# Example usage with your existing code:
conservative_bounds = compute_vertex_conservative_bounds(
    model, 
    vbd_integrator, 
    colision_detector, 
    gamma_p=0.5
)
# 
# # The conservative bounds can then be used for collision detection
# # as described in equation (27): ||x_v - x_v^prev|| <= b_v

Module __main__ cfef323 load on device 'cuda:0' took 21.35 ms


AttributeError: 'tuple' object has no attribute 'ctype'

In [21]:
import polyscope as ps
import numpy as np
import warp as wp

def visualize_conservative_bounds(model, conservative_bounds, mesh, collision_detector=None):
    """
    Visualize the conservative bounds results using Polyscope.
    
    Args:
        model: The simulation model
        conservative_bounds: Array of conservative bounds for each vertex
        mesh: Original mesh object with vertices and faces
        collision_detector: Optional collision detector for additional visualizations
    """
    
    # Initialize Polyscope
    ps.init()
    ps.set_up_dir("z")
    ps.set_front_dir("-y")
    
    # Convert warp arrays to numpy if needed
    if hasattr(conservative_bounds, 'numpy'):
        bounds_np = conservative_bounds.numpy()
    else:
        bounds_np = np.array(conservative_bounds)
    
    # Get mesh vertices and faces
    vertices = mesh.vertices
    faces = mesh.faces
    
    # Register the mesh
    ps_mesh = ps.register_surface_mesh("cloth_mesh", vertices, faces)
    
    # Visualize conservative bounds as vertex colors
    ps_mesh.add_scalar_quantity("Conservative Bounds", bounds_np, 
                               defined_on='vertices', 
                               cmap='viridis',
                               enabled=True)
    
    # Add additional visualizations if collision detector is provided
    if collision_detector is not None:
        # Visualize vertex-triangle collision distances
        if hasattr(collision_detector, 'vertex_colliding_triangles_min_dist'):
            vertex_tri_dist = collision_detector.vertex_colliding_triangles_min_dist
            if hasattr(vertex_tri_dist, 'numpy'):
                vertex_tri_dist_np = vertex_tri_dist.numpy()
            else:
                vertex_tri_dist_np = np.array(vertex_tri_dist)
            
            ps_mesh.add_scalar_quantity("Vertex-Triangle Min Distance", vertex_tri_dist_np,
                                       defined_on='vertices',
                                       cmap='plasma',
                                       enabled=False)
        
        # Visualize edge-edge collision distances (mapped to vertices)
        if hasattr(collision_detector, 'edge_colliding_edges_min_dist'):
            edge_edge_dist = collision_detector.edge_colliding_edges_min_dist
            if hasattr(edge_edge_dist, 'numpy'):
                edge_edge_dist_np = edge_edge_dist.numpy()
            else:
                edge_edge_dist_np = np.array(edge_edge_dist)
            
            # Map edge distances to vertices (take minimum of adjacent edges)
            vertex_edge_dist = np.full(len(vertices), np.inf)
            for edge_idx in range(len(edge_edge_dist_np)):
                # This assumes model.edge_indices contains edge vertex pairs
                if hasattr(model, 'edge_indices'):
                    edge_verts = model.edge_indices[edge_idx * 2:(edge_idx + 1) * 2]
                    for v_idx in edge_verts:
                        vertex_edge_dist[v_idx] = min(vertex_edge_dist[v_idx], edge_edge_dist_np[edge_idx])
            
            vertex_edge_dist[vertex_edge_dist == np.inf] = 0.0
            ps_mesh.add_scalar_quantity("Edge-Edge Min Distance", vertex_edge_dist,
                                       defined_on='vertices',
                                       cmap='coolwarm',
                                       enabled=False)
    
    # Add some mesh analysis
    # Compute vertex normals for better visualization
    vertex_normals = compute_vertex_normals(vertices, faces)
    ps_mesh.add_vector_quantity("Vertex Normals", vertex_normals, 
                               defined_on='vertices',
                               enabled=False,
                               length=0.1)
    
    # Add mesh edges for better structure visualization
    ps_mesh.set_edge_width(0.5)
    ps_mesh.set_edge_color([0.2, 0.2, 0.2])
    
    # Create a summary statistics display
    print_statistics(bounds_np, collision_detector)
    
    # Show the visualization
    ps.show()

def compute_vertex_normals(vertices, faces):
    """Compute vertex normals from face normals."""
    vertex_normals = np.zeros_like(vertices)
    
    for face in faces:
        # Compute face normal
        v0, v1, v2 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        normal = np.cross(v1 - v0, v2 - v0)
        normal = normal / (np.linalg.norm(normal) + 1e-8)
        
        # Add to vertex normals
        vertex_normals[face[0]] += normal
        vertex_normals[face[1]] += normal
        vertex_normals[face[2]] += normal
    
    # Normalize
    norms = np.linalg.norm(vertex_normals, axis=1, keepdims=True)
    vertex_normals = vertex_normals / (norms + 1e-8)
    
    return vertex_normals

def print_statistics(bounds_np, collision_detector=None):
    """Print summary statistics about the conservative bounds."""
    print("\n" + "="*50)
    print("CONSERVATIVE BOUNDS ANALYSIS")
    print("="*50)
    
    print(f"Total vertices: {len(bounds_np)}")
    print(f"Bounds range: [{bounds_np.min():.6f}, {bounds_np.max():.6f}]")
    print(f"Mean bound: {bounds_np.mean():.6f}")
    print(f"Std deviation: {bounds_np.std():.6f}")
    
    # Percentile analysis
    percentiles = [10, 25, 50, 75, 90, 95, 99]
    print(f"\nPercentile analysis:")
    for p in percentiles:
        print(f"  {p}th percentile: {np.percentile(bounds_np, p):.6f}")
    
    # Identify vertices with extreme bounds
    tight_threshold = np.percentile(bounds_np, 5)
    loose_threshold = np.percentile(bounds_np, 95)
    
    tight_vertices = np.where(bounds_np <= tight_threshold)[0]
    loose_vertices = np.where(bounds_np >= loose_threshold)[0]
    
    print(f"\nVertices with tight bounds (<= {tight_threshold:.6f}): {len(tight_vertices)}")
    print(f"Vertices with loose bounds (>= {loose_threshold:.6f}): {len(loose_vertices)}")
    
    if collision_detector is not None:
        print(f"\nCollision detection info:")
        if hasattr(collision_detector, 'vertex_colliding_triangles_min_dist'):
            vtri_dist = collision_detector.vertex_colliding_triangles_min_dist
            if hasattr(vtri_dist, 'numpy'):
                vtri_dist_np = vtri_dist.numpy()
                print(f"  Vertex-triangle distances: [{vtri_dist_np.min():.6f}, {vtri_dist_np.max():.6f}]")
        
        if hasattr(collision_detector, 'edge_colliding_edges_min_dist'):
            edge_dist = collision_detector.edge_colliding_edges_min_dist
            if hasattr(edge_dist, 'numpy'):
                edge_dist_np = edge_dist.numpy()
                print(f"  Edge-edge distances: [{edge_dist_np.min():.6f}, {edge_dist_np.max():.6f}]")
    
    print("="*50)

def create_interactive_visualization(model, conservative_bounds, mesh, collision_detector=None):
    """
    Create an interactive visualization with multiple views and controls.
    """
    # Initialize Polyscope
    ps.init()
    ps.set_up_dir("z")
    ps.set_front_dir("-y")
    
    # Convert arrays
    bounds_np = conservative_bounds.numpy() if hasattr(conservative_bounds, 'numpy') else np.array(conservative_bounds)
    
    # Register mesh
    ps_mesh = ps.register_surface_mesh("cloth_mesh", mesh.vertices, mesh.faces)
    
    # Add multiple scalar quantities for comparison
    ps_mesh.add_scalar_quantity("Conservative Bounds", bounds_np, 
                               defined_on='vertices', cmap='viridis')
    
    # Log-scale bounds for better visualization of small values
    log_bounds = np.log10(bounds_np + 1e-8)
    ps_mesh.add_scalar_quantity("Log Conservative Bounds", log_bounds, 
                               defined_on='vertices', cmap='plasma')
    
    # Normalized bounds (0-1 scale)
    normalized_bounds = (bounds_np - bounds_np.min()) / (bounds_np.max() - bounds_np.min())
    ps_mesh.add_scalar_quantity("Normalized Bounds", normalized_bounds, 
                               defined_on='vertices', cmap='coolwarm')
    
    # Create point cloud for vertices with extreme bounds
    tight_threshold = np.percentile(bounds_np, 10)
    loose_threshold = np.percentile(bounds_np, 90)
    
    tight_vertices = np.where(bounds_np <= tight_threshold)[0]
    loose_vertices = np.where(bounds_np >= loose_threshold)[0]
    
    if len(tight_vertices) > 0:
        tight_points = mesh.vertices[tight_vertices]
        ps.register_point_cloud("Tight Bound Vertices", tight_points, 
                               color=[1.0, 0.0, 0.0], point_radius=0.01)
    
    if len(loose_vertices) > 0:
        loose_points = mesh.vertices[loose_vertices]
        ps.register_point_cloud("Loose Bound Vertices", loose_points, 
                               color=[0.0, 1.0, 0.0], point_radius=0.01)
    
    # Set initial view
    ps_mesh.set_transparency(0.8)
    ps_mesh.set_material("wax")
    
    print("Interactive visualization ready!")
    print("Use the GUI to:")
    print("- Toggle between different scalar quantities")
    print("- Adjust transparency and material")
    print("- View tight/loose bound vertices")
    print("- Analyze the conservative bounds distribution")
    
    ps.show()



In [22]:
# Example usage with your existing code:
# After computing conservative bounds:
visualize_conservative_bounds(model, conservative_bounds, mesh, colision_detector)


NameError: name 'conservative_bounds' is not defined